In [1]:
import numpy as np
import subprocess
import time
import copy
from utils.opt_utils import *
import os
import h5py
import pickle
from qutip import *
from scipy.optimize import minimize, basinhopping, brute

In [2]:
wc_A = 4.069814 * (10**9) * 2 * np.pi  # cavity A frequency
wc_B = 6.096062 * (10**9) * 2 * np.pi  # cavity A frequency
wa =  5.325 * (10**9) * 2 * np.pi  # atom frequency
dt_A = np.abs(wc_A - wa) / (2 * np.pi)
dt_B = np.abs(wc_B - wa) / (2 * np.pi)
chi_A = 0.00215 * (10**9) * 2 * np.pi
chi_B = 0.00544 * (10**9) * 2 * np.pi
g_A = np.sqrt(chi_A * dt_A) * 2 * np.pi  # coupling strength w/ cavity A
g_B = np.sqrt(chi_B * dt_B) * 2 * np.pi  # coupling strength w/ cavity B

gamma = 333333.333        # atom dissipation rate
kappa_A = 10000       # cavity A dissipation rate
kappa_B = 10000       # cavity B dissipation rate

temp_q = 0.01        # avg number of thermal bath excitation for qubit
temp_A = 0.04        # avg number of thermal bath excitation for cavity A
temp_B = 0.05        # avg number of thermal bath excitation for cavity B

In [3]:
cavity_dims = 8

# Cost function
def cost_q_e(final_expect, final_dm):
    # print(final_expect[0])
    return(final_expect[0])

def cost_qA_g1(final_expect, final_state):
    return np.power(np.abs(final_state.full()[1][0]), 2)

def cost_qAB_g11(final_expect, final_dm):
    return np.power(np.abs(final_dm.full()[cavity_dims + 1][0]), 2)

def cost_qAB_g11_dm(final_expect, final_state):
    return np.power(np.abs(final_state[cavity_dims + 1][0]), 2)

def cost_qAB_g11_n(final_expect, final_dm):
    noise = (np.random.rand(1)[0] * 0.10) - 0.05
    return np.abs(final_dm.full()[cavity_dims + 1][0]) + noise

def cost_01_ge_entangle(final_expect, final_state):
    return np.power(np.abs(final_state[0][0]) + np.abs(final_state[cavity_dims + 1][0]), 2) / 2

def cost_dihydrogen_entangle(final_expect, final_state):
    return np.power(
        np.abs(final_state[0][0]) + 
        np.abs(final_state[2][0]) + 
        np.abs(final_state[1][0]) + 
        np.abs(final_state[2 * cavity_dims][0]) +
        np.abs(final_state[cavity_dims][0]) +
        np.abs(final_state[cavity_dims + 1]) +
        (2 * np.abs(final_state[0][0] * final_state[2][0])) +
        (2 * np.abs(final_state[2 * cavity_dims][0] * final_state[1][0]))
    , 2) / 10

In [21]:
# ========== OPTIONS ========== #
max_segs = 20
us = 0.000001
time_start = 0.0000000
time_stop = 2 * us
init_qamp = 942
init_camp = 4465
n_steps = 501

num_drives = 3
num_cavities = 2
# cavity_dims = 8
state_sizes = [2, cavity_dims]
state_vals = [0.0, 0.5, 0.5]
init_freqs = [wa, wc_A, wc_B]
sim_options = Options()
element_freqs = [wa, wc_A, wc_B]
drive_elem_nums = [0, 1, 2]
output_cost_func = cost_dihydrogen_entangle
elements = "qAB"
start_split_num = 20
n_seg_jump = 1
verbose = False

load_pulse_dir = r'C:\Users\Wang_Lab\Documents\GitLab\quantum_control_rl_server\examples\sim_entangle_3E_interp_classical'
load_times_file = r'opt_SNAP_times_1g.txt'
load_amps_file = r'opt_SNAP_amps_1g.txt'
use_loaded_data = False

save_dir = r'C:\Users\Wang_Lab\Documents\GitLab\quantum_control_rl_server\examples\sim_entangle_3E_interp_classica\save_data'
hdf5_name = time.strftime('%Y%m%d-%H%M%S.h5')
epochs = 1000
epochs_per_seg = 500
train_batch_size = 20
maximum_iterations_start = 1
maximum_iterations_step = 1
qubit_amp_scale = 4
cavity_amp_scale = 4
freq_scale = 0.0005
# ========== OPTIONS ========== #

t_arr = np.linspace(time_start, time_stop, n_steps)

t_step = (time_stop - time_start) / n_steps

sim_options.store_final_state = True

qscale = np.array([])
cscale = np.array([])
time_scale = np.array([])
for i in range(2 * start_split_num):
    qscale = np.append(qscale, np.array([-init_qamp * qubit_amp_scale, init_qamp * qubit_amp_scale]))
    cscale = np.append(cscale, np.array([-init_camp * cavity_amp_scale, init_camp * cavity_amp_scale]))
    cscale = np.append(cscale, np.array([-init_camp * cavity_amp_scale, init_camp * cavity_amp_scale]))
    
for i in range(2 * start_split_num * num_drives):
    time_scale = np.append(time_scale, np.array([time_start, time_stop]))

sm, a_A, a_B, sx, sz = reg_ops(num_cavities + 1, cavity_dims)
drive_freqs = np.array(init_freqs)

gammas = [gamma, kappa_A, kappa_B]
temps = [temp_q, temp_A, temp_B]
c_ops = [] # gen_c_ops(elements, [sm, a_A, a_B, sx, sz], gammas, temps)

# Operators used in Hamiltonian
drive_ops = [sm.dag(), sm, a_A.dag(), a_A, a_B.dag(), a_B]
element_ops = [sz, a_A.dag() * a_A]
H_0 = -(chi_A * a_A.dag() * a_A * sz) - (chi_B * a_B.dag() * a_B * sz)

eval_ops = [sm.dag() * sm, a_A.dag() * a_A, a_B.dag() * a_B]

t_segs, amp_segs = setup_interp_segs(2 * num_drives, time_start, time_stop, init_qamp)

amp_segs[:2, :] = init_qamp
amp_segs[2:, :] = init_camp

# Setup initial state
init_state = tensor((basis(state_sizes[0], 0) * np.sqrt(1 - state_vals[0])) + (basis(state_sizes[0], 1) * np.sqrt(state_vals[0])), (basis(state_sizes[1], 0) * np.sqrt(1 - state_vals[1])) + (basis(state_sizes[1], 1) * np.sqrt(state_vals[1])), (basis(state_sizes[1], 0) * np.sqrt(1 - state_vals[1])) + (basis(state_sizes[1], 1) * np.sqrt(state_vals[1])))

amp_segs = np.reshape(amp_segs, (num_drives * 2, int(len(amp_segs.flatten()) / (num_drives * 2))))

t_segs = np.reshape(t_segs, (num_drives * 2, int(len(t_segs.flatten()) / (num_drives * 2))))

t_segs = t_segs[:, 1:-1]

for i in range(start_split_num - 1):
    t_segs, amp_segs = split_segs_flat(interp_time_wrapper(t_segs, time_start, time_stop), interp_amp_wrapper(amp_segs))
    t_segs = t_segs[:, 1:-1]

# Flips occasional bits to give it a random 
flip_bits = np.array([[False, False, True, True]])
flip_mask = np.array([np.repeat(flip_bits, np.ceil(start_split_num / 4), axis=0).flatten()[:start_split_num]])
flip_mask = np.repeat([flip_mask], num_drives * 2, axis=1)[0]

amp_segs[flip_mask] *= -1

if use_loaded_data:
    print("Loading data")
    load_pulse_times = np.loadtxt(str(os.path.join(load_pulse_dir, load_times_file)))
    load_pulse_amps = np.loadtxt(str(os.path.join(load_pulse_dir, load_amps_file)))
    
    t_segs = np.array(load_pulse_times)
    amp_segs = np.array(load_pulse_amps)

# Create blank history arrays for storing optimal / past values
time_hist = []
amp_hist = []
freq_hist = []
cost_hist = []

In [24]:
# Run vqe, etc
vmax = np.vectorize(max)
vmin = np.vectorize(min)

hdf5_start_index = 0
start_segs = start_split_num
for i in range(max_segs):

    client_args = [num_drives, drive_ops, element_freqs, H_0, init_state, t_arr, eval_ops, sim_options, output_cost_func, verbose, drive_freqs, drive_elem_nums]
    
    print(np.shape(qscale))
    
    amp_bounds = np.append(qscale, cscale)
    bounds = np.append(amp_bounds, time_scale)
    
    x0 = np.append(amp_segs.flatten(), t_segs.flatten())

    # res = minimize(sim_interp_cost_eval, x0=x0, method='Powell', args=client_args, options={"maxiter": ((i * maximum_iterations_start) + maximum_iterations_step)}, bounds=bounds)
    
    break
    
    sol = res.x
    
    # Update amps and times for next round
    opt_amps = sol[:2 * num_drives * (start_split_num)]
    
    opt_times = sol[2 * num_drives * start_split_num:]
    
    opt_result = res.fun

    # updates amplitudes and frequencies with optimized values and reshape
    amp_segs = np.array(opt_amps)
    amp_segs = np.reshape(amp_segs, (num_drives * 2, int(len(amp_segs.flatten()) / (num_drives * 2))))

    t_segs = np.array(opt_times)
    t_segs = np.reshape(t_segs, (num_drives * 2, int(len(amp_segs.flatten()) / (num_drives * 2))))

    print(f'================')
    print(f'num segs: {i + start_segs} ')
    print(f'opt_amps: {amp_segs}')
    print(f'opt_times: {t_segs}')
    print(f'opt_result: {opt_result}')

    # save values to history arrays
    time_hist.append(interp_time_wrapper(t_segs, time_start, time_stop))
    amp_hist.append(interp_amp_wrapper(amp_segs))
    cost_hist.append(opt_result)

    np.savez(r'run_data\\' + hdf5_name[:-3] + "-" + str(i) + ".npz", time=time_hist[-1], amp=amp_hist[-1], cost=cost_hist[-1])


    for i in range(2 * n_seg_jump):
        qscale.append(qubit_amp_scale)
        cscale.append(cavity_amp_scale)
        cscale.append(cavity_amp_scale)

    # split segments and return to start of loop
    if (i < max_segs - 1):
        for i in range(2):
            qscale = np.append(qscale, np.array([-init_qamp * qubit_amp_scale, init_qamp * qubit_amp_scale]))
            cscale = np.append(cscale, np.array([-init_camp * cavity_amp_scale, init_camp * cavity_amp_scale]))
            cscale = np.append(cscale, np.array([-init_camp * cavity_amp_scale, init_camp * cavity_amp_scale]))
            
        for i in range(2 * num_drives):
            time_scale = np.append(time_scale, np.array([time_start, time_stop]))

(80,)
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-3768.0
3768.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17860.0
17860.0
-17